In [1]:
from amplpy import AMPL, add_to_path
add_to_path(r"C:\ampl")
import nltk
nltk.download('words')
from nltk.corpus import words
from tqdm import tqdm
from time import sleep

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
def writeData(s1, s2, filename = "ampl_files/input.dat"):
    
    file = open(filename, "w")
    file.writelines([
            f"param n:={len(s1)};\n",
            f"param m:={len(s2)};\n",
            "param gap_p:=1;\n",
            "param mis_p:=1;\n",
            "param r:=-1;\n",
            "param a :=\n"
        ])
    
    a = []
    for c1 in s1:
      row = []
      for c2 in s2:
        row.append(1 if c1 == c2 else 0)
      a.append(row)
    
    for i in range(len(a)):
      for j in range(len(a[i])):
        file.write(f"{i+1} {j+1} {a[i][j]} \n")
    
    file.write(";\n")
    file.close()

In [3]:
def tooDifferent(a, b):
    for c in a:
        if c in b:
            return False
    return True

In [4]:
def computeScore(target, word, inputfile):
    ampl = AMPL()
    best = -100, ""
    writeData(target, word, inputfile)

    ampl.eval(r''' 
        reset;
        model ./ampl_files/SequenceAlignment.mod;
        data ''' + inputfile + ''';
        option solver cbc;
        option solver_msg 0; 
        solve >ampl_files/out.txt;
        ''')

    score = -round(ampl.get_value("obj"))
    return score

In [ ]:
english_words = words.words()

text = "helo"

if text not in english_words:

    filtered = filter(lambda w: abs(len(w)-len(text)) <= 0.5*len(text), english_words )
    filtered = filter(lambda w: not tooDifferent(text, w), list(filtered))
    filtered = list(filtered)

    n=len(filtered)

    best = -100, ""
    for i in tqdm(range(0, n), colour="#00ffff", desc ="Progress: "):
        score = computeScore(text, filtered[i], "ampl_files/input0.dat")
        if score > best[0]:
            best = score, filtered[i]
            if score >= len(text)-2:
                break
    print(f"Best match: {best}")
else:
    print(f"Best match: {text}")


Progress:   2%|█                                                                 | 443/28032 [02:12<2:26:58,  3.13it/s]